In [ ]:
#Import the packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#from sklearn import preprocessing 
%matplotlib inline

In [ ]:
df = pd.read_csv('garments_worker_productivity.csv')
df

In [ ]:
#transform actual productivity to categorical data
tmp_condition = df['actual_productivity'] > 0.735091097
df.loc[tmp_condition, 'actual_productivity'] = 'Satisfactory'
df.loc[~tmp_condition, 'actual_productivity'] = 'Unsatisfactory'

In [ ]:
# Convert the categorical data to Numerical data
df['quarter'].replace(['Quarter1', 'Quarter2', 'Quarter3', 'Quarter4', 'Quarter5'],
                       [1, 2, 3, 4, 5], inplace=True)
df['day'].replace(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
                        [1, 2, 3, 4, 5, 6, 7], inplace=True)
df['actual_productivity'].replace(['Satisfactory', 'Unsatisfactory'],
                       [1, 0,], inplace=True)

df

In [ ]:
#Visualize the data

# Create the histogram (separate plots for each series)
plt.hist(df["targeted_productivity"], alpha=0.5, label="Target", color='red')
plt.hist(df["actual_productivity"], alpha=0.5, label="Actual", color='black')
 
# Adding labels and a title
plt.ylabel('actual_productivity')
plt.xlabel('targeted_productivity')
plt.title('Histogram Graph showing the targeted productivity against the actual productivity')

# Add legend
plt.legend()
 
# Display the plot
plt.show()

In [ ]:
# Defining the Input and Output
df_input = df.drop(columns = ['department', 'date', 'wip',
                              'incentive', 'idle_time', 'idle_men', 'no_of_style_change', 'actual_productivity'], axis=1)
df_output = df['actual_productivity']

In [ ]:
df_input

In [ ]:
df_output

In [ ]:
# Next is the Normalization of the data
#Use The min-max feature scaling method to normalize the data
# copy the data 
df_min_max_scaled = df_input.copy()

# List of columns to scale
columns_to_scale = ['quarter', 'day', 'team', 'targeted_productivity', 'smv', 'over_time', 'no_of_workers']

# Apply Min-Max normalization
for column in columns_to_scale:
    df_min_max_scaled[column] = (df_min_max_scaled[column]
                                 - df_min_max_scaled[column].min()) / (df_min_max_scaled[column].max() 
                                                                       - df_min_max_scaled[column].min())

# View normalized data
display(df_min_max_scaled)

# ANN From Scratch

In [ ]:
inputs = df_min_max_scaled.to_numpy()
output = df_output.to_numpy()

In [ ]:
inputs

In [ ]:
output

In [ ]:
# Sigmoid Function and Derivative
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

# Generate random input and output data (replace with actual data)
#inputs(1197, 7)
#output(1197, 1)

# Initialization of Weights
weights_0 = 2 * np.random.random((7, 16)) - 1
weights_1 = 2 * np.random.random((16, 1)) - 1

# Training Parameters
epochs = 1000
learning_rate = 0.1
error = []

# Training Loop
for epoch in range(epochs):
    input_layer = inputs 
    sum_synapse0 = np.dot(input_layer, weights_0) 
    hidden_layer = sigmoid(sum_synapse0)
 
    sum_synapse1 = np.dot(hidden_layer, weights_1) 
    output_layer = sigmoid(sum_synapse1)
 
    error_output_layer = output - output_layer 
    average = np.mean(abs(error_output_layer))

   #prints and stores error every 100 epoch
    if epoch % 100 == 0:
        print('Epoch: ' + str(epoch + 1) + ' Error: ' + str(average))
        error.append(average)

In [ ]:
plt.xlabel('Number of Epochs')
plt.ylabel('Error')
plt.title('Plot showing the results')
plt.plot(error)
plt.show()

In [ ]:
output_layer

In [ ]:
preds = pd.DataFrame(output_layer, columns = ['outputs'])
preds['prediction'] = 1
preds.loc[preds['outputs'] < 0.5, 'prediction'] = 0
preds

In [ ]:
actual = pd.DataFrame(output)
both = pd.merge(actual,preds, left_index = True, right_index = True)
both